# Classification de texte optimisée avec TF-IDF et SVM (GridSearch)

Ce notebook détaille un pipeline avancé de classification de texte utilisant le prétraitement linguistique avec SpaCy, la vectorisation TF-IDF, un SVM linéaire, et une optimisation des hyperparamètres par GridSearchCV. Chaque étape est expliquée, du nettoyage à la génération du fichier de soumission.

---

## Concepts clés

- **TF-IDF** : pondère chaque mot selon sa fréquence dans le document et sa rareté dans le corpus.
- **SVM linéaire** : cherche l'hyperplan qui sépare au mieux les classes dans l'espace vectoriel.
- **GridSearchCV** : recherche systématique des meilleurs hyperparamètres par validation croisée.
- **Macro-F1** : mesure la performance globale en équilibrant l'importance de chaque classe.

---

In [ ]:
# Importation des bibliothèques nécessaires
import json
import pandas as pd
import re
import spacy
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, make_scorer, f1_score

## 1. Chargement et préparation des données

On charge les données d'entraînement et les labels, puis on fusionne les deux. Les descriptions manquantes sont remplacées par une chaîne vide.

In [ ]:
with open('train.json','r',encoding='utf-8') as f:
    train_data = json.load(f)
df = pd.DataFrame(train_data).fillna({'description':""})
labels = pd.read_csv('train_label.csv')
df = df.merge(labels, on='Id')

## 2. Nettoyage et lemmatisation des textes

On utilise SpaCy pour nettoyer et lemmatiser les textes :
- Suppression du HTML et des URLs
- Mise en minuscules
- Lemmatisation (réduction à la racine)
- Suppression des stopwords et tokens non alphabétiques

Ce prétraitement réduit la dimensionnalité et améliore la qualité des features pour le modèle.

In [ ]:
nlp = spacy.load('en_core_web_sm')
def clean_and_tokenize(text):
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+', ' ', text).lower().strip()
    doc = nlp(text)
    return " ".join(tok.lemma_ for tok in doc if tok.is_alpha and not tok.is_stop)

df['Clean'] = df['description'].apply(clean_and_tokenize)

## 3. Séparation train/validation

On sépare les données en un ensemble d'entraînement (80%) et de validation (20%) pour évaluer la capacité de généralisation du modèle. On stratifie pour conserver la proportion des classes.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df['Clean'], df['Category'],
    test_size=0.2, stratify=df['Category'], random_state=42
)

## 4. Pipeline et optimisation par GridSearchCV

On construit un pipeline combinant la vectorisation TF-IDF et un SVM linéaire. On utilise GridSearchCV pour tester plusieurs combinaisons d'hyperparamètres :
- ngram_range, min_df, max_df pour TF-IDF
- C et class_weight pour le SVM

La validation croisée (cv=5) permet de sélectionner les meilleurs paramètres selon le score macro-F1.

In [ ]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LinearSVC(dual=False, max_iter=5000))
])

param_grid = {
    "tfidf__ngram_range": [(1,1), (1,2)],
    "tfidf__min_df": [3, 5, 10],
    "tfidf__max_df": [0.8, 0.9, 1.0],
    "clf__C": [0.01, 0.1, 1.0, 10.0, 100.0],
    "clf__class_weight": [None, "balanced"]
}

scorer = make_scorer(f1_score, average="macro")

grid = GridSearchCV(
    pipeline, param_grid,
    cv=5, n_jobs=-1, scoring=scorer, verbose=1
)
grid.fit(X_train, y_train)

## 5. Évaluation sur la validation

On prédit les classes sur l'ensemble de validation et on affiche un rapport de classification (précision, rappel, F1-score). Cela permet de vérifier la performance réelle du modèle optimisé.

In [ ]:
y_val_pred = grid.predict(X_val)
print("=== Rapport sur le set de validation ===")
print(classification_report(y_val, y_val_pred))

## 6. Prédiction sur les données de test

On applique le même nettoyage et la même vectorisation aux textes de test, puis on prédit la catégorie pour chaque texte.

In [ ]:
with open('test.json','r',encoding='utf-8') as f:
    test_data = json.load(f)
test_df = pd.DataFrame(test_data).fillna({'description':""})
test_df['Clean'] = test_df['description'].apply(clean_and_tokenize)
preds = grid.predict(test_df['Clean'])

## 7. Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
template = pd.read_csv('template_submissions.csv')
mapping = dict(zip(test_df['Id'], preds))
template['Category'] = template['Id'].map(mapping)
template.to_csv('submission_svm_tuned.csv', index=False)
print(" Soumission enregistrée : submission_svm_tuned.csv")

---

## Concepts mathématiques et conclusion

- **TF-IDF** : $tfidf(w, d) = tf(w, d) \times \log\left(\frac{N}{df(w)}\right)$
- **SVM** : maximise la marge entre les classes dans l'espace vectoriel.
- **GridSearchCV** : sélectionne les meilleurs hyperparamètres par validation croisée.
- **Macro-F1** : équilibre la performance sur toutes les classes, même rares.

Ce pipeline montre comment combiner prétraitement, vectorisation, optimisation et classification pour obtenir un modèle robuste et performant sur des textes.